This notebook covers the basic of creating UDTFs for manipulating a `GeoMultiPolygon`: A set of one or more associated lines, each of two or more points.

For example: `MULTIPOLYGON(((0 0,4 0,4 4,0 4,0 0),(1 1,2 1,2 2,1 2,1 1)), ((-1 -1,-1 -2,-2 -2,-2 -1,-1 -1)))`

For the API, check the RBC ReadTheDocs page:
* [`Column<GeoMultiPolygon>`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.ColumnGeoMultiPolygon.html#rbc.heavydb.ColumnGeoMultiPolygon)
* [`GeoMultiPolygon`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.GeoMultiPolygon.html#rbc.heavydb.GeoMultiPolygon)

In [93]:
import warnings; warnings.filterwarnings('ignore')

### Connect to the HeavyDB server

In [94]:
# NBVAL_IGNORE_OUTPUT
from rbc.heavydb import RemoteHeavyDB
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

GeoMultiLineString requires HeavyDB 7.0 or newer

In [95]:
heavydb.version[:3]

(7, 0, 0)

### Load test data

In [96]:
from util import load_test_data
from rbc.tests import _MultiPolygonTestTable
table_name = 'mpolygon_table'
load_test_data(heavydb, _MultiPolygonTestTable, table_name)

List of multilinestrings in `mline_table`

In [97]:
import pandas as pd
descr, result = heavydb.sql_execute(f'select * from {table_name}')
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

,mp1,mp2,mp3,mp4
0,"MULTIPOLYGON (((1 2,3 4,5 6,7 8,9 10,1 2),(1 2...","MULTIPOLYGON (((0 0,4.99999995576218 0.0,4.999...","MULTIPOLYGON (((0 0,6 0,6 6,0 6,0 0),(3 3,3 2,...","MULTIPOLYGON (((0 0,7 0,7 7,0 7,0 0),(4 4,4 2,..."
1,"MULTIPOLYGON (((0 0,5 0,5 5,0 5,0 0)))","MULTIPOLYGON (((0 0,5.99999998044223 0.0,5.999...","MULTIPOLYGON (((0 0,7 0,7 7,0 7,0 0)))","MULTIPOLYGON (((0 0,4 0,4 4,0 4,0 0)))"
2,"MULTIPOLYGON (((1 2,3 4,5 6,7 8,9 10,1 2),(2 3...","MULTIPOLYGON (((0 0,4.99999995576218 0.0,4.999...","MULTIPOLYGON (((0 0,6 0,6 6,0 6,0 0),(3 3,3 2,...","MULTIPOLYGON (((0 0,7 0,7 7,0 7,0 0),(4 4,4 2,..."
3,None,None,None,None


### Define a function that operate on GeoMultiPolygon

Function `get_polygon` takes a `Column<GeoMultiPolygon>` as input and construct a polygon.

In [98]:
@heavydb("int32(TableFunctionManager, Column<Z>, int64, OutputColumn<K>)",
            Z=['GeoMultiPolygon'], K=['GeoPolygon'], devices=['cpu'])
def get_polygon(mgr, mpolygons, n, polygons):
    size = len(mpolygons)
    mgr.set_output_item_values_total_number(0, mpolygons.get_n_of_values())
    mgr.set_output_row_size(size)
    for i in range(size):
        if mpolygons.is_null(i):
            polygons.set_null(i)
        else:
            sz = len(mpolygons[i])
            if n < 1 or n > sz:
                polygons.set_null(i)
            else:
                polygons.set_item(i, mpolygons[i][n - 1])
    return size

In [99]:
from pprint import pprint

In [100]:
descr, result = heavydb.sql_execute(f'select mp1 from {table_name}')
pprint(list(result))

[('MULTIPOLYGON (((1 2,3 4,5 6,7 8,9 10,1 2),(1 2,3 4,2 3,1 2)))',),
 ('MULTIPOLYGON (((0 0,5 0,5 5,0 5,0 0)))',),
 ('MULTIPOLYGON (((1 2,3 4,5 6,7 8,9 10,1 2),(2 3,1 2,3 4,2 3),(9 10,7 8,5 6,9 '
  '10)))',),
 (None,)]


In [102]:
query = (f'''
    SELECT * FROM TABLE(get_polygon(
        cursor(SELECT mp1 from {table_name}),
        1
    ))
''')

descr, result = heavydb.sql_execute(query)
pprint(list(result))

[('POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(1 2,2 3,3 4,1 2))',),
 ('POLYGON ((0 0,5 0,5 5,0 5,0 0))',),
 ('POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(2 3,3 4,1 2,2 3),(9 10,5 6,7 8,9 '
  '10))',),
 (None,)]
